# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
price_data = dd.read_parquet(parquet_files, engine="pyarrow")
price_data.head()


Price,Date,Adj Close,Close,High,Low,Open,Volume,Year
Ticker,,,,,,,,
A,2000-01-03,NaN,43.382835,47.562950,40.596089,47.449974,4674353.0,2000
A,2000-01-04,NaN,40.068867,41.499898,39.014422,41.047991,4765083.0,2000
A,2000-01-05,NaN,37.583401,40.068878,36.340666,39.918241,5758642.0,2000
A,2000-01-06,NaN,36.152363,37.357444,35.022601,37.131491,2534434.0,2000
A,2000-01-07,NaN,39.165070,39.729950,35.549834,35.587491,2819626.0,2000


For each ticker and using Dask, do the following:
Add lags for variables Close and Adj_Close.
Add returns based on Close:
returns: (Close / Close_lag_1) - 1
Add the following range:
hi_lo_range: this is the day's High minus Low.
Assign the result to dd_feat.
(4 pt)

In [18]:
df = dd.read_parquet(parquet_files)

df_lags = df.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Close_lag_2 = x['Adj Close'].shift(1))
)

dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(returns = (x['Close'] / x['Close_lag_1']) - 1,
                       hi_lo_range = x['High'] - x['Low'])
)
dd_feat.head()

/var/folders/nc/pbf0pj7114jg0dltbbtyzj0m0000gn/T/ipykernel_62976/3938836255.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_lags = df.groupby('Ticker', group_keys=False).apply(
/var/folders/nc/pbf0pj7114jg0dltbbtyzj0m0000gn/T/ipykernel_62976/3938836255.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(


Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Close_lag_2,returns,hi_lo_range
Ticker,,,,,,,,,,,,


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [20]:
# Write your code below.

dd_feat = dd_feat.compute()
dd_feat['rolling_mean'] = dd_feat['returns'].rolling(10).mean()
dd_feat


Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Close_lag_2,returns,hi_lo_range,rolling_mean
Ticker,,,,,,,,,,,,,
DOV,2020-01-02,NaN,108.841797,108.841797,107.668349,108.171255,937500.0,2020,NaN,NaN,NaN,1.173448,NaN
DOV,2020-01-03,NaN,107.826668,107.957050,106.755664,107.212009,587400.0,2020,108.841797,NaN,-0.009327,1.201386,NaN
DOV,2020-01-06,NaN,107.835968,107.891844,106.643897,106.820840,754200.0,2020,107.826668,NaN,0.000086,1.247947,NaN
DOV,2020-01-07,NaN,108.134010,108.422713,106.709112,107.035068,810700.0,2020,107.835968,NaN,0.002764,1.713601,NaN
DOV,2020-01-08,NaN,108.385460,108.581033,107.398277,107.957060,848400.0,2020,108.134010,NaN,0.002325,1.182756,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2024-12-24,NaN,NaN,NaN,NaN,NaN,NaN,2024,NaN,NaN,NaN,NaN,NaN
CTLT,2024-12-26,NaN,NaN,NaN,NaN,NaN,NaN,2024,NaN,NaN,NaN,NaN,NaN
CTLT,2024-12-27,NaN,NaN,NaN,NaN,NaN,NaN,2024,NaN,NaN,NaN,NaN,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?


Yes, converting to Pandas made sense here. Dask doesn’t handle .rolling() inside groupby().apply() well, so doing it in Pandas was the simplest and most efficient way.

+ Would it have been better to do it in Dask? Why?


Not necessarily — unless the dataset was too big for memory, Pandas was the faster and easier choice.



(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.